In [147]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import networkx as nx
import gc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tsp-aco/tsp_100_5
/kaggle/input/tsp-aco/tsp_144_1
/kaggle/input/tsp-aco/tsp_226_1
/kaggle/input/tsp-aco/tsp_136_1
/kaggle/input/tsp-aco/tsp_657_1
/kaggle/input/tsp-aco/tsp_195_1
/kaggle/input/tsp-aco/tsp_5_1
/kaggle/input/tsp-aco/tsp_1748_1
/kaggle/input/tsp-aco/tsp_124_1
/kaggle/input/tsp-aco/tsp_783_1
/kaggle/input/tsp-aco/tsp_1655_1
/kaggle/input/tsp-aco/tsp_493_1
/kaggle/input/tsp-aco/tsp_11849_1
/kaggle/input/tsp-aco/tsp_299_1
/kaggle/input/tsp-aco/tsp_150_2
/kaggle/input/tsp-aco/tsp_4461_1
/kaggle/input/tsp-aco/tsp_159_1
/kaggle/input/tsp-aco/tsp_574_1
/kaggle/input/tsp-aco/tsp_2152_1
/kaggle/input/tsp-aco/tsp_2392_1
/kaggle/input/tsp-aco/tsp_1400_1
/kaggle/input/tsp-aco/tsp_1323_1
/kaggle/input/tsp-aco/tsp_400_1
/kaggle/input/tsp-aco/tsp_1060_1
/kaggle/input/tsp-aco/tsp_2319_1
/kaggle/input/tsp-aco/tsp_575_1
/kaggle/input/tsp-aco/tsp_1577_1
/kaggle/input/tsp-aco/tsp_107_1
/kaggle/input/tsp-aco/tsp_1817_1
/kaggle/input/tsp-aco/tsp_100_3
/kaggle/input/tsp-aco/tsp_129

In [148]:
from IPython.display import FileLink
import csv
def submission_generation(filename, str_output):
    os.chdir(r'/kaggle/working')
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        for item in str_output:
            writer.writerow(item)
    return  FileLink(filename)

In [149]:
import math
from collections import namedtuple

In [150]:
Point = namedtuple("Point", ['x', 'y'])

def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

In [151]:
def nearestNode(p, iNode):
    # Cogemos como primera aproximacion del nodo mas cercano el siguiente, 
    # si lo hubiera en caso contrario el anterior
    if iNode + 1 >= len(p):
        indexNearNode = iNode - 1
        valueNearNode = length(p[iNode], p[iNode - 1])
    else:
        indexNearNode = iNode + 1
        valueNearNode = length(p[iNode], p[iNode + 1])
    # Recorremos el los puntos desde el actual para delante y cogemos el menor que haya por delante
    for i in range(iNode + 1, len(p)):
        if p[i].x - p[iNode].x >= valueNearNode:
            break
        if math.fabs(p[iNode].y - p[i].y) >= valueNearNode:
            continue
        if length(p[iNode], p[i]) < valueNearNode:
            indexNearNode = i
            valueNearNode = length(p[iNode], p[i])
    # Recorremos los puntos del actual a 0 si hubiera algun punto mas cercano que la solucion
    # que ya tengamos lo cogemos
    for i in reversed(range(0, iNode)):
        if p[iNode].x - p[i].x >= valueNearNode:
            break
        if math.fabs(p[iNode].y - p[i].y) >= valueNearNode:
            continue
        if length(p[iNode], p[i]) < valueNearNode:
            indexNearNode = i
            valueNearNode = length(p[iNode], p[i])

    return indexNearNode

In [153]:
def check_solution(edges, nNodes, points):
    ar = np.zeros(nNodes)
    it = edges[0][0]
    ar[edges[0][0]] = 1
    # Recorremos cada arista apuntando los nodos ya recorridos 
    # si un nodo se recorriera dos veces devolveriamos false
    for i in range(0, len(edges) - 1):
        if edges[i][0] == it or edges[i][1] == it:
            if edges[i][0] == it:
                it = edges[i][1]
                if ar[it] == 1:
                    return False
                ar[it] = 1
            else:
                it = edges[i][0]
                if ar[it] == 1:
                    return False
                ar[it] = 1
    # Tratamos la ultima arista aparte si esta no conectara el ultimo nodo con el primero devolvemos false
    if edges[len(edges)-1][1] != edges[0][0]:
        return False
    # Si alguna nodo se recorriera 0 veces devolveriamos false
    for i in ar:
        if i == 0:
            return False
    cost = 0
    # Recorremos las aristas acumulando el coste de un nodo a otro
    for i in range(0, len(edges)):
        cost += length(points[edges[i][0]], points[edges[i][1]])

    return cost


In [154]:
def greedy(p):
    # res guardara las aristas que forman nuestro resultado
    res = []
    indArr = [i for i in range(0, len(p))]
    pos = nearestNode(p, 0) - 1
    res.append([0, pos + 1])
    p.pop(0)
    indArr.pop(0)
    # Recorremos los nodos de p saltando de uno a otro hasta que solo quede 1 por recorrer
    # Quitamos de p aquellos ya visitados
    while len(p) > 1:
        auxPos = pos
        # Obtenemos la posicion del nodo mas cercano al nodo actual
        pos = nearestNode(p, pos)
        res.append([indArr[auxPos], indArr[pos]])
        if auxPos < pos:
            pos -= 1
        p.pop(auxPos)
        indArr.pop(auxPos)
    # Unimos el último nodo con el primero
    res.append([res[len(res) - 1][1], res[0][0]])
    return res

In [155]:
def multiGraph(p):
    # Creamos un multigrafo le insertamos todos los nodos de p y despues le insertamos todas las 
    # aristas para crear un grafo completo.
    G = nx.MultiGraph()
    for i in range(0, len(p)):
        G.add_node(i)
    for i in range(0, len(p)):
        for j in range(i + 1, len(p)):
            G.add_edge(i, j, weight=(length(p[i], p[j])))
    return G

In [156]:
def oddNodes(sortEdges, p):
    count = np.zeros(len(p))
    # Contamos las veces que se recorre cada nodo en el arbol de expansion minima
    for i in sortEdges:
        count[i[0]] += 1
        count[i[1]] += 1
    
    odd = nx.Graph()
    oddAr = []
    # Obtenemos el array con aquellos nodos con grado impar
    for i in range(0, len(count)):
        if count[i] % 2 != 0:
            oddAr.append(i)
            
    # Generamos el grafo con los nodos con grado impar y a partir de el creamos un grafo completo
    odd.add_nodes_from(oddAr)
    
    for i in range(0, len(oddAr)):
        for j in range(i + 1, len(oddAr)):
            odd.add_edge(oddAr[i], oddAr[j], weight=(1/length(p[oddAr[i]], p[oddAr[j]])))
    
    return odd

In [157]:
def christofides(p):
    G = multiGraph(p) 

    # Mediante networkx conseguimos el arbol de expansion minima.
    minTree = nx.minimum_spanning_tree(G)
    # Liberamos memoria
    G = 1
    gc.collect()
    
    sortEdges = sorted(minTree.edges(data=True))
    # Obtenemos el grafo con los nodos de grado impar
    odd = oddNodes(sortEdges, p)
    
    # Obtenemos el emparejamiento perfecto
    oddMatch = nx.max_weight_matching(odd)
    # Unimos estas aristas al arbol minimo
    for i in oddMatch:
        minTree.add_edge(i[0], i[1], weight=(length(p[i[0]], p[i[1]])))

    # Obtemeos el circuito euleriano
    euPath = list(nx.eulerian_circuit(minTree))
    euPath = [list(item) for item in euPath]

    aux = np.zeros(len(p))
    i = 0
    # Finalmente quitamos aquellas aristas que pasen por un nodo ya visitado
    while i < len(euPath):
        if aux[euPath[i][0]] != 1:
            aux[euPath[i][0]] = 1
            i += 1
        else:
            euPath[i-1][1] = euPath[i][1]
            euPath.pop(i)
            
    return euPath

In [158]:
def solve_it(input_data):
    lines = input_data.split('\n')
    nodeCount = int(lines[0])
    points = []
    for i in range(1, nodeCount + 1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
        
    # Ordenamos primero por las y's y despues por las x's
    # Esto nos permite tener los puntos mas a la izquierda primero y si tuvieran
    # el mismo valor de x iria antes los de menos valor de y
    points.sort(key=lambda p:  p[1])
    points.sort(key=lambda p:  p[0])
    
    print(len(points)," \tPathCost:" ,end = ' ')
    
    p = points.copy()
    # A partir de 3700 nodos el tiempo de espera es mayor a 1 h y 
    # hemos decidido para estos aplicar el algoritmo greedy
    if len(points) > 0:
        solution = greedy(p)
    else:
        solution = christofides(p)

    # calculate the length of the tour
    obj = 0
    for i in range(0, len(solution)):
        obj += length(points[solution[i][0]], points[solution[i][1]])

    return points,solution, obj

In [159]:
str_output = [["Filename","Value"]]
for dirname, _, filenames in os.walk('/kaggle/input/tsp-aco'):
    for filename in filenames:
        full_name = dirname+'/'+filename
        with open(full_name, 'r') as input_data_file:
            input_data = input_data_file.read()
            p, output, value = solve_it(input_data)
            # Check solution devuelve false si el resultado es incorrecto
            cost = check_solution(output,len(p), p)
            if cost != False and cost == value:
                print(cost)
                str_output.append([filename,str(value)])
            else:
                print("Resultado incorrecto")
                    

100  	PathCost: 25221.774636380087
144  	PathCost: 61725.23999867418
226  	PathCost: 112542.35448729275
136  	PathCost: 119306.36032483357
657  	PathCost: 62275.1337948178
195  	PathCost: 2761.9573600471717
5  	PathCost: 4.0
1748  	PathCost: 435797.31212928944
124  	PathCost: 71553.14962881827
783  	PathCost: 11815.591456627251
1655  	PathCost: 72862.94567432944
493  	PathCost: 41887.20728645508
11849  	PathCost: 1131897.345956751
299  	PathCost: 62559.50433057508
150  	PathCost: 38844.54417297562
4461  	PathCost: 229044.14299356524
159  	PathCost: 54707.61287398136
574  	PathCost: 47000.48778768433
2152  	PathCost: 77906.9094152049
2392  	PathCost: 474644.24752486113
1400  	PathCost: 26551.22962345794
1323  	PathCost: 330054.3681674009
400  	PathCost: 19367.920278671132
1060  	PathCost: 290341.52536185354
2319  	PathCost: 265913.85269384424
575  	PathCost: 8232.758150616452
1577  	PathCost: 26519.09375134279
107  	PathCost: 50230.57634608763
1817  	PathCost: 71421.83710583928
100  	Pa

KeyboardInterrupt: 

In [ ]:
str_output

In [ ]:
submission_generation('sample_submission_non_sorted.csv', str_output)

In [ ]:
reader = csv.reader(open("sample_submission_non_sorted.csv"))
sortedlist = sorted(reader, key=lambda row: row[0], reverse=False)

In [ ]:
submission_generation('sample_submission.csv', sortedlist)